In [3]:
! pip install watchdog

     |████████████████████████████████| 92kB 818kB/s eta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/61/1d/d0/04cfe495619be2095eb8d89a31c42adb4e42b76495bc8f784c
  Stored in directory: /home/jovyan/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built watchdog pathtools


In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 20;

<IPython.core.display.Javascript object>

In [4]:
from OutputProcessing import process_output_files

In [5]:
process_output_files('../responses/', 1000)

IntProgress(value=21, description='Iter 21/1000', max=1000)

'Mean: 0.397'

,title,url,sentiment
0,Page #14,url,0.988
1,Page #12,url,0.894
2,Page #13,url,0.734
3,Page #5,url,0.672
4,Page #8,url,0.643
5,Page #15,url,0.494
6,Page #17,url,0.443
7,Page #0,url,0.442
8,Page #9,url,0.379
9,Page #7,url,0.372
